# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

![Farmer in a field](farmer_in_a_field.jpg)

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will apply machine learning to build a multi-class classification model to predict the type of `"crop"`, while using techniques to avoid multicollinearity, which is a concept where two or more features are highly correlated.

In [28]:
# All required libraries are imported here for you.
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import f1_score

# Load the dataset
crops = pd.read_csv("soil_measures.csv")

In [29]:
#Seeing data types and missing values to ensure availability for model
print(crops.dtypes)
print(crops.isna().any())

N         int64
P         int64
K         int64
ph      float64
crop     object
dtype: object
N       False
P       False
K       False
ph      False
crop    False
dtype: bool


In [30]:
#Importing label encoder to categorize crop to allow prediction
from sklearn.preprocessing import LabelEncoder

#Initializing Label Encoder
le = LabelEncoder()

#Encoding crop into category
crops["crop"] = le.fit_transform(crops["crop"])

#Viewing New Column
print(crops["crop"])

0       20
1       20
2       20
3       20
4       20
        ..
2195     5
2196     5
2197     5
2198     5
2199     5
Name: crop, Length: 2200, dtype: int64


In [31]:
#Preparing x and y variables for modelling
X = crops.drop("crop", axis=1).values
y = crops["crop"].values

#Viewing shape of variables
print(X.shape)
print(y.shape)

(2200, 4)
(2200,)


In [32]:
#Creating train/test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
#Importing LinearRegression and f1_score from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

#Initializing model with multi_class and limiting iterations at 2000
model = LogisticRegression(multi_class = "multinomial", max_iter=2000)

#Creating models list
models = []

#Creating scores list
score_list = []

#Iterating model in for loop to append model from each feature into list
for col in range(X_train.shape[1]):
    #Defining training values
    X_train_col = X_train[:, col].reshape(-1, 1)
    
    #Defining test values 
    X_test_col = X_test[:, col].reshape(-1, 1)
    
    #Fitting model
    model.fit(X_train_col, y_train)
    
    #Predicting values
    y_pred = model.predict(X_test_col)
    
    #Calculating f1 score
    f1 = f1_score(y_test, y_pred, average="macro")
    
    #Appending predictions in models list
    models.append(y_pred)
    
    #Appending f1 score in score list
    score_list.append(f1)

print(score_list)

[0.10993898082576807, 0.11919303408608646, 0.208028683149612, 0.04671180311261425]


In [34]:
#Import numpy
import numpy as np

#Doing correlation analysis to check for multicollinearity
df_corr = crops.drop("crop", axis=1)
corr_matrix = df_corr.corr()

#Viewing the matrix
print(corr_matrix)

#P and K have a correlation of 0.73, which indicates linearity amongst them. Since K had a higher score in the test, I opted to drop P as a feature.

           N         P         K        ph
N   1.000000 -0.231460 -0.140512  0.096683
P  -0.231460  1.000000  0.736232 -0.138019
K  -0.140512  0.736232  1.000000 -0.169503
ph  0.096683 -0.138019 -0.169503  1.000000


In [35]:
#Selecting final_features based on performance
final_features = ["N", "K", "ph"]
X = crops[final_features]

#Redeploying train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Initializing the model
log_reg = LogisticRegression(multi_class = "multinomial", max_iter=2000)

#Fitting the model
log_reg.fit(X_train, y_train)

#Predicting categories of crop
y_pred = log_reg.predict(X_test)

#Calcultaing F1 Score
model_performance = f1_score(y_test, y_pred, average="macro")

print(model_performance)

0.5578553115462431
